In [1]:
import findspark
findspark.init('/home/yaniv/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lin_reg').getOrCreate()
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler

In [3]:
assembler = VectorAssembler(inputCols =  ['Age',
                                         'Total_Purchase',
                                         'Account_Manager',
                                         'Years',
                                         'Num_Sites'],
                            outputCol =  'features')

data = assembler.transform(df)

In [4]:
from pyspark.sql.functions import desc

In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'features']

In [6]:
data.select('Account_Manager').show()

+---------------+
|Account_Manager|
+---------------+
|              0|
|              0|
|              0|
|              0|
|              0|
|              0|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              0|
|              0|
|              1|
+---------------+
only showing top 20 rows



In [7]:
final = data.select('features','Churn')

In [8]:
final.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [9]:
from pyspark.ml.classification import LogisticRegression

In [10]:
lg = LogisticRegression(labelCol='Churn')

In [11]:
train,test = final.randomSplit([0.7,0.3])

In [12]:
lg = lg.fit(train)

In [13]:
pred = lg.transform(test)

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol='rawPrediction',
    labelCol='Churn')

In [15]:
# Atry to understand What probaility first parameter affect prediction

In [21]:
selected_data = pred.select('Churn','prediction','probability')
selected_data.show()

+-----+----------+--------------------+
|Churn|prediction|         probability|
+-----+----------+--------------------+
|    0|       0.0|[0.98703043136736...|
|    0|       0.0|[0.99795515935595...|
|    0|       0.0|[0.82494196318059...|
|    0|       0.0|[0.97255385749575...|
|    0|       0.0|[0.98220281152113...|
|    0|       0.0|[0.98633340139850...|
|    0|       0.0|[0.99433381267477...|
|    0|       0.0|[0.98028389113573...|
|    0|       0.0|[0.99693868397736...|
|    0|       0.0|[0.90151631191574...|
|    0|       0.0|[0.90281042864631...|
|    0|       0.0|[0.99846576161247...|
|    0|       0.0|[0.99006399820007...|
|    1|       0.0|[0.71509738880673...|
|    0|       0.0|[0.96053485100944...|
|    0|       0.0|[0.98778347860463...|
|    0|       0.0|[0.57286571758841...|
|    0|       0.0|[0.88514727379522...|
|    0|       0.0|[0.83000918943575...|
|    0|       0.0|[0.86955563291711...|
+-----+----------+--------------------+
only showing top 20 rows



In [17]:
pred.select('probability').collect()[0][0][0]

0.9870304313673616

In [18]:
prob_value=[]
for i in range(pred.select('probability').count()):
    prob_value.append(pred.select('probability').collect()[i][0][0])

In [26]:
#must find a way to reconstruct a column with probability measurable parameter

In [27]:
# devide data to several parts due to its probability

In [ ]:
prob_over96 = selected_data.filter('probability>0.96')
prob_over85 = selected_data.filter('probability>0.85')
prob_between70and85 = selected_data.filter((selected_data['probability']<85)&(selected_data['probability']>70))
prob_under60 = selected_data.filter('probability<0.6')

In [ ]:
#evaluate each data 

In [ ]:
evaluator.evaluate(prob_over96)
evaluator.evaluate(prob_over85)
evaluator.evaluate(prob_between70and85)
evaluator.evaluate(prob_under60)

In [ ]:
#general evaluation

In [34]:
evaluator.evaluate(pred)

0.8788850174216032